In [1]:
import logging
import os

import wiki as w 
 
from localgensim.gensim2.models import word2vec 

from tqdm import tqdm

In [ ]:
w.download_wiki_dump('en', '/home/manni/data/wiki/en-28102022.bz2')

[2022-09-28 12:48:57,898] Downloading https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles-multistream.xml.bz2 to /home/manni/data/wiki/en-28102022.bz2
  7%|███████████▏                                                                                                                                             | 1.52M/20.7M [05:18<1:09:19, 4.62kKB/s]

In [2]:
import sys
sys.path.append("../../imports/")
import saver as sv

In [3]:
logging.basicConfig(format='[%(asctime)s] %(message)s', level=logging.INFO)
os.makedirs('data/', exist_ok=True)

In [ ]:
sentences = sv.load("wiki_sentences_wnet") #New

In [ ]:
found = False
for sent in sentences:
    for i,word in enumerate(sent):
        if 'rapid#' in word:
            print(sent[i-10:i],sent[i:i+11])
            found = True
            break
    if found:
        break

In [ ]:
word2desc = sv.load('word2desc')

In [ ]:
word2desc['rapid'][0]

In [ ]:
word2desc['rapid']

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
rapid = wn.synsets('rapid')
print(rapid)

In [ ]:
for i in rapid:
    print(i.hyponyms())

In [ ]:
for i in rapid:
    print(i.hypernyms())

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
glove = '/home/manni/embs/gloveW2V.6B.300d.txt'
w2v = '/home/manni/embs/w2v.bin.gz'
spx = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300_mixture.txt'
wnet ='/home/manni/embs/en_wiki_wnet_epoch5_300_w10_exclusive.txt'

In [ ]:
w2v_model = KeyedVectors.load_word2vec_format(w2v, binary=True)

In [ ]:
glove_model = KeyedVectors.load_word2vec_format(glove, binary=False)

In [ ]:
spx_model = KeyedVectors.load_word2vec_format(spx, binary=False)

In [ ]:
wnet_model = KeyedVectors.load_word2vec_format(wnet, binary=False)

In [ ]:
#model = glove_model

In [ ]:
#model = w2v_model

In [ ]:
#model = spx_model

In [ ]:
model = wnet_model

In [ ]:
def get_matrix(model,vocab):
    matrix = list()
    for word in vocab:
        if word in model.vocab:
            vec = model.get_vector(word)
        else:
            vec = np.zeros(model.vector_size)
        matrix.append(vec)
    return np.asarray(matrix)

In [ ]:
def get_clusters(model,word):
    sets = word2desc[word]
    clusters_ = list()
    n_clusters_ = list()
    for words in sets:
        vocab = list(words)
        matrix = get_matrix(model,vocab)
        db = DBSCAN(eps=0.4,min_samples=2,metric='cosine').fit(matrix)
        labels = db.labels_
        clusters = dict()
        for i,c in enumerate(labels):
            if c == -1:
                continue
            if c in clusters:
                clusters[c].append(vocab[i])
            else:
                clusters[c] = [vocab[i]]
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        clusters_.append(clusters)
        n_clusters_.append(n_clusters)
    return clusters_,n_clusters_

In [ ]:
get_clusters(model,'rapid')

In [ ]:
for words in word2desc['rapid']:
    words = list(words)
    for i in range(len(words)):
        for j in range(i+1,len(words)):
            print(words[i],words[j],model.similarity(words[i],words[j]))
    print('------')

In [ ]:
model.similarity(['waterway','river'],'current')

In [ ]:
model.doesnt_match({'current', 'fast', 'part', 'river', 'waterway'})

In [ ]:
v = model.get_vector('waterway')+model.get_vector('river')

In [ ]:
model.cosine_similarities(v,[model.get_vector('current')])

In [ ]:
model

In [ ]:
a = ['current#3','current#2','current#1','current','current#0']
b = ['river','river#0']
c = ['waterway','waterway#0','waterway#1']

In [ ]:
for word in a:
    for _word in b:
        print(word,_word,model.similarity(word,_word))
    print('-------')

In [ ]:
def avg_sim(w1,w2,model,verbose=False,maxsim=False):
    range_ = list()
    vocab = model.vocab
    a = list()
    for word in vocab:
        if word.startswith(w1+'#'):
            a.append(word)
    if w1 in vocab:
        a.append(w1)
    b = list()
    for word in vocab:
        if word.startswith(w2+'#'):
            b.append(word)
    if w2 in vocab:
        b.append(w2)
    div = len(a)*len(b)
    if div == 0:
        return 0, range_
    sims = list()
    for i in a:
        for j in b:
            sim = model.similarity(i,j)
            range_.append(sim)
            if verbose:
                print(i,j,sim)
            sims+=[sim]
    if maxsim:
        return max(sims), range_
    else:
        return sum(sims)/div, range_

In [ ]:
avg_sim('rapid','fast',model,verbose=True,maxsim=True)

# tags

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
doc = nlp("the raft floated downstream on the current")
for token in doc:
    #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
    print(token.pos_)

# contexts

In [4]:
sentences = sv.load("wiki_sentences_wnet")

In [5]:
for sent in sentences:
    print(sent[:100])
    break

FileNotFoundError: [Errno 2] No such file or directory: '/home/manni/data/wiki/enwiki-20211120-pages-articles-multistream.xml.bz2'

In [ ]:
examples = list()
def get_contexts(word,sentences):
    i = 1
    for sent in sentences:
        for j,_word in enumerate(sent):
            if word == _word.split('#'):
                examples.append(sent[j-10:j]+sent[j:j+11])
                i+=1
            elif word == _word.split('§'):
                examples.append(sent[j-10:j]+sent[j:j+11])
                i+=1
            if i>10:
                break
        if i>10:
                break
    return examples

In [ ]:
get_contexts('rapid',sentences)